<font size=10>**Making new layers via subclassing**</font>

参考(Making new layers and models via subclassing)[https://keras.io/guides/making_new_layers_and_models_via_subclassing/]
在模型中添加自定义层
+ 构建我们自定义的softmax2层（实现softmax功能，包括全连接层,softmax只是一个激活（或者变换））
+ 加载一个训练好的lenet模型
+ 用softmax2层来替换lenet模型的最后一层softmax

In [1]:
import tensorflow as tf
print(f"tf verion = {tf.__version__}")

import tensorflow_model_optimization as tfmot
from tensorflow.keras.layers import InputLayer,Reshape,Conv2D,MaxPool2D,Flatten,Dense,Dropout
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow import keras

tf verion = 2.2.0


*解决GPU内存不足报错，对GPU进行按需分配

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# 构建自定义层softmax2

In [3]:
class Softmax2(keras.layers.Layer):
    def __init__(self, units=10):
        super(Softmax2, self).__init__()
        self.units = units
        self.dense = keras.layers.Dense(
            units, kernel_regularizer=tf.keras.regularizers.l2(1e-3)
        )
        
    def call(self, inputs):
        return tf.nn.softmax(self.dense(inputs))

    def get_config(self):
        return {"units": self.units}

# 加载Lenet网络
## 加载数据集

In [4]:
# 加载 MNIST 数据集
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 归一化输入图片，这样每个像素的值都在[0, 1]之间
x_train, x_test = x_train / 255.0, x_test / 255.0

# 扩张输入数据维度[height, width, channels(depth)]
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

## 加载预训练Lenet网络

In [5]:
pretrained_model = load_model("./model/lenet_normal.hdf5")

## 评估预训练网络

In [6]:
pretrained_model.summary()
print("==> evaluate")
base_model_accuracy = pretrained_model.evaluate(x_test, y_test, verbose=2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 6)         150       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 16)        2400      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               94200     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

## 替换最有一层softmax为softmax2

In [7]:
# 替换我们自定义的softmax2层 
def apply_custom_layers(layer):
    if layer.name == "dense_2":
        return Softmax2(units=10) 
    return layer

customzed_model = tf.keras.models.clone_model(
    pretrained_model, clone_function=apply_custom_layers,)

## 评估使用自定义层的网络

In [8]:
customzed_model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
customzed_model.summary()

customzed_model.set_weights(pretrained_model.get_weights())
print("==> evaluate")
custom_model_accuracy = customzed_model.evaluate(x_test, y_test, verbose=2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 6)         150       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 16)        2400      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               94200     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

In [9]:
customzed_model.save("./model/lenet_custom.hdf5")